<a href="https://colab.research.google.com/github/Abdulmuj33b/TechCrush_Capstone_TeamStark/blob/main/Heart_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 - Exploratory Data Analysis (EDA)

- Loads `heart.xls` from `/mnt/data`
- Cleans and summarizes the dataset
- Produces visualizations (displayed inline) and saves them to `images/`
- Performs simple statistical tests (t-test for numeric vs target, chi-square for categorical vs target)
- Prints short, insights after each analysis step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 7.0 MB/s eta 0:00:00


In [ ]:
# Standard imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, confusion_matrix, classification_report, RocCurveDisplay, PrecisionRecallDisplay
from imblearn.over_sampling import SMOTE
import joblib
from IPython.display import display

# Make plots look nicer
%matplotlib inline
sns.set(style='whitegrid')

# Paths - consolidated
DATA_PATH = '/content/MyDrive/heart-disease-project/data/heart.xls'
CLEAN_PATH = '/content/MyDrive/heart-disease-project/data/cleaned.heart.csv'
IMAGES_DIR = os.path.join('/content/MyDrive/heart-disease-project/images')
os.makedirs(IMAGES_DIR, exist_ok=True)
print('Images will be saved to:', IMAGES_DIR)

Images will be saved to: /content/MyDrive/heart-disease-project/images


In [ ]:
# 1. Load the data
# It appears the file is actually a CSV despite the .xls extension
df = pd.read_csv(DATA_PATH)
print('Initial shape:', df.shape)
display(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/MyDrive/heart-disease-project/data/heart.xls'

In [ ]:
from google.colab import drive
# Removed unmount as it's not available in this version
# try:
#   drive.unmount('/content/drive')
# except ValueError:
#   pass # Already unmounted or not mounted
drive.mount('/content/gdrive')

ValueError: Mountpoint must not already contain files

In [ ]:
!pip install xlrd

In [ ]:
# 2. Basic cleaning function (lowercase cols, strip whitespace, replace empty strings)
def basic_clean(df):
    df = df.copy()
    df.columns = [str(c).strip().lower().replace(' ', '_') for c in df.columns]
    df = df.replace(r'^\s*$', np.nan, regex=True)
    return df

df = basic_clean(df)
print('After basic cleaning shape:', df.shape)
display(df.head())

In [ ]:
# Identify duplicate rows
duplicate_rows = df[df.duplicated()]

# Display the head of the duplicate rows
print("Head of duplicate rows:")
display(duplicate_rows.head())

# Print the total number of duplicate rows
print("\nTotal number of duplicate rows:", duplicate_rows.shape[0])

## Why We Kept Duplicates in This Health Dataset

We did not remove duplicate rows in this health dataset because:

*   **Multiple visits/events:** Patients can have multiple records representing different visits or medical events.
*   **Different observations:** Multiple valid observations might exist for the same patient at a given time.
*   **Data nuances:** The data collection process can result in seemingly duplicate, but valid, entries.

Keeping these rows preserves the full patient history and avoids losing important information.

In [ ]:
# Ensure target exists and is binary
if 'target' not in df.columns:
    raise KeyError("The dataset must contain a 'target' column with 0/1 values indicating heart disease.")
print('Target unique values:', df['target'].unique())


## 3. Quick summary & missingness


In [ ]:
display(df.info())
display(df.describe(include='all').T)

# Missing values summary
missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing>0]
print('\nColumns with missing values:\n')
print(missing)


## 4. Class distribution (target)


In [ ]:
ax = df['target'].value_counts().sort_index().plot(kind='bar', color=['blue', 'red'])
ax.set_xticklabels(['No Disease (0)','Disease (1)'], rotation=0)
ax.set_ylabel('Count')
ax.set_title('Target class distribution')
plt.tight_layout()
fn = os.path.join(IMAGES_DIR, 'target_distribution.png')
plt.savefig(fn, dpi=150)
plt.show()
print('\nQuick-insight:')
counts = df['target'].value_counts().sort_index()
if counts.shape[0]==2:
    pct_disease = counts.loc[1] / counts.sum()
    print(f"Proportion with heart disease: {pct_disease:.2%} (class=1).")
    pct_no_disease = counts.loc[0] / counts.sum()
    print(f"Proportion without heart disease: {pct_no_disease:.2%} (class=0).")
    print("This indicates whether class imbalance handling may be needed.")
else:
    print('Target is not binary or unexpected unique values.')

## 5. Identifying numeric and categorical columns


This is crucial because it determines how we process, visualize, and prepare data for modeling. Different data types require different methods.

In [ ]:
num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()
print('Numeric columns:', num_cols)
print('Categorical columns:', cat_cols)


## 6. Univariate analysis — numeric features

it helps us to understand:

    Distribution patterns of individual variables (normal, skewed, uniform)

    Central tendency through mean, median, and mode

    Data spread via range, variance, and standard deviation

    Presence of outliers that may need special treatment

    Overall characteristics of each feature in isolation

This foundational understanding informs subsequent data preprocessing and guides more complex multivariate analysis.

For each numeric feature we will plot a histogram and boxplot, save them, and compute skewness.

In [ ]:
for col in num_cols:
    fig, axes = plt.subplots(1,2, figsize=(12,4))
    sns.histplot(df[col].dropna(), kde=True, ax=axes[0])
    axes[0].set_title(f'Histogram of {col}')
    sns.boxplot(x=df[col], ax=axes[1])
    axes[1].set_title(f'Boxplot of {col}')
    plt.tight_layout()
    fn = os.path.join(IMAGES_DIR, f'{col}_hist_box.png')
    plt.savefig(fn, dpi=150)
    plt.show()
    skew = df[col].skew()
    print(f"{col} — skewness: {skew:.2f}")


## 7. Bivariate analysis  (Numeric vs target)
We now compare distributions of numeric features grouped by `target` and run t-tests (or Mann-Whitney if non-normal).

Essentially, we are checking if the distribution or values of a numeric feature are significantly different between the two target classes (heart disease or no heart disease). This gives us insights into which numeric features might be important predictors of the target.

In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu

for col in num_cols:
    data0 = df[df['target']==0][col].dropna()
    data1 = df[df['target']==1][col].dropna()
    # plot
    plt.figure(figsize=(6,4))
    sns.boxplot(x='target', y=col, data=df)
    plt.title(f'{col} by target')
    fn = os.path.join(IMAGES_DIR, f'{col}_by_target_box.png')
    plt.savefig(fn, dpi=150)
    plt.show()
    # choose test based on normality (Shapiro) and sample size
    use_mw = False
    try:
        if len(data0) >= 3 and len(data1) >= 3:
            p0 = stats.shapiro(data0.sample(min(5000, len(data0))))[1] if len(data0) <= 5000 else 1.0
            p1 = stats.shapiro(data1.sample(min(5000, len(data1))))[1] if len(data1) <= 5000 else 1.0
            if p0 < 0.05 or p1 < 0.05:
                use_mw = True
    except Exception:
        use_mw = True
    if use_mw:
        stat, p = mannwhitneyu(data0, data1, alternative='two-sided')
        test_name = 'Mann-Whitney U'
    else:
        stat, p = ttest_ind(data0, data1, nan_policy='omit')
        test_name = 'T-test'
    print(f"{col}: {test_name} p-value = {p:.4f}")
    if p < 0.05:
        print(f"  -> Quick insight: Significant difference in {col} between classes (p<{0.05}).")
    else:
        print(f"  -> Quick insight: No significant difference detected for {col} (p={p:.3f}).")


## 8. Correlation matrix (numeric features)

This helps us see which features are buddies or rivals, and if any are strongly linked to our goal (target)

In [ ]:
corr = df[num_cols].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', square=True)
plt.title('Correlation matrix (numeric features)')
fn = os.path.join(IMAGES_DIR, 'correlation_matrix.png')
plt.savefig(fn, dpi=150, bbox_inches='tight')
plt.show()


## 9. Correlation of features with target (point-biserial for numeric)

Here is our way to measure the relationship between the continuous (numeric) variables and the binary (two-category) variable, like our target (disease or no disease).
A higher absolute value means the numeric feature is a stronger indicator of whether someone has the disease or not.

In [ ]:
from scipy.stats import pointbiserialr
corrs = []
for col in num_cols:
    try:
        r, p = pointbiserialr(df['target'].fillna(0), df[col].fillna(df[col].median()))
        corrs.append({'feature': col, 'r': r, 'p': p})
    except Exception as e:
        pass
corr_df = pd.DataFrame(corrs).sort_values('r', key=abs, ascending=False)
display(corr_df)
plt.figure(figsize=(6,4))
sns.barplot(x='r', y='feature', data=corr_df)
plt.title('Point-biserial correlation with target')
fn = os.path.join(IMAGES_DIR, 'feature_target_correlation.png')
plt.savefig(fn, dpi=150, bbox_inches='tight')
plt.show()
print('\nQuick insight: Features with largest absolute correlation (top 5):')
print(corr_df.head(5).to_string(index=False))


## 10. Simple pairwise plots for top features

This will be our visual way to see potential interactions and separation power of the top predictors.

In [ ]:
top_feats = corr_df['feature'].head(4).tolist()
# Remove 'target' from top_feats if it's there, as it will be used for hue
if 'target' in top_feats:
    top_feats.remove('target')

if len(top_feats) >= 2:
    # Pass the features to plot on axes and the target for hue
    g = sns.pairplot(df[top_feats + ['target']].dropna(), vars=top_feats, hue='target', corner=True)
    g.fig.suptitle('Pairwise Plots of Top Features by Target', y=1.02) # Add a main title
    fn = os.path.join(IMAGES_DIR, 'pairplot_top_features.png')
    plt.savefig(fn, dpi=300, bbox_inches='tight') # Increase DPI for potentially clearer saved image
    plt.show()
else:
    print('Not enough top numeric features for pairplot.')

Based on the above scatter plots of top key features (oldpeak, exang, cp, thalach):

    Clear visual separation exists between heart disease cases (red) and healthy individuals (blue)

    Distinct clustering patterns show these features effectively differentiate between disease states

    The observed separation validates earlier correlation findings

    These visual patterns confirm these variables' importance for predicting heart disease



## 11. Outlier detection using IQR (InterQuartile Range) method and counts

Identifying outliers allows will us to understand their nature and decide on appropriate strategies, such as investigating their source, removing it, transforming the data, or using models that are more robust to outliers.


In [ ]:
outlier_counts = {}
for col in num_cols:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    oc = df[(df[col] < lower) | (df[col] > upper)].shape[0]
    outlier_counts[col] = oc
outlier_df = pd.DataFrame.from_dict(outlier_counts, orient='index', columns=['outlier_count']).sort_values('outlier_count', ascending=False)
display(outlier_df)
print('\nQuick insight: Columns with many outliers may need robust scaling or capping before we start modeling.')


## 12. Missingness visualization


In [ ]:
ms = df.isnull().sum()
if ms.sum() == 0:
    print('No missing values detected in the dataset.')
else:
    ms = ms[ms>0].sort_values(ascending=False)
    ms.plot.barh(figsize=(6, max(3, len(ms)*0.4)))
    plt.title('Missing values per column')
    fn = os.path.join(IMAGES_DIR, 'missing_values.png')
    plt.savefig(fn, dpi=150, bbox_inches='tight')
    plt.show()


## 13. Save a cleaned version of the dataset for modeling


In [ ]:
import pandas as pd
from google.colab import files

# Save the cleaned dataset directly in the current working directory

# Assuming your cleaned DataFrame is named df
df.to_csv(CLEAN_PATH, index=False)

print('✅ Saved cleaned data to:', CLEAN_PATH)

# Trigger file download to your local machine
# Removed files.download(CLEAN_PATH) as the file is saved directly to Google Drive

## 14. Final Summary of EDA


In [ ]:
print('--- EDA SUMMARY ---')
print(f'Total rows: {df.shape[0]}, Total columns: {df.shape[1]}')
print('\nTop numeric features correlated with target:')
display(corr_df.head(10))
print('\nColumns with missing values:')
display(df.isnull().sum()[df.isnull().sum()>0])
print('\nColumns with most outliers:')
display(outlier_df.head(10))
print('\nRecommendations:')
print(' - Handle class imbalance if disease prevalence is low (resampling or class weights).')
print(' - Consider log-transform or robust scaling for skewed features.')
print(' - Impute missing values (median for numeric, mode for categorical) or use model-based imputers.')
print(' - Use SHAP/feature importance after modeling to get clinical explanations.')


## 15. Data Validation using pandera schema

    Ensures data quality and catches issues early

    Defines data contracts for structure, types, and value ranges

    Prevents analysis errors from invalid/missing data

    Validates business rules (e.g., realistic medical ranges)

    Supports reliable analytics and model building



We are maintaining trustworthy analysis results and production-ready data workflows.

Pandera Validation Turns Our Data Quality from "hope" into guarantee

In [ ]:
!pip install pandera

In [ ]:

# Data validation with pandera schema
try:
    import pandera.pandas as pa
    from pandera import Column, DataFrameSchema
except Exception as e:
    raise ImportError('pandera not installed. Please install with `pip install pandera`') from e

# Generate schema from df dtypes
if 'df' not in globals():
    print('df not found. Please ensure df is loaded before running this cell.')
else:
    col_map = {}
    for col, dtype in df.dtypes.items():
        if pd.api.types.is_integer_dtype(dtype):
            col_map[col] = pa.Column(pa.Int, nullable=df[col].isnull().any())
        elif pd.api.types.is_float_dtype(dtype):
            col_map[col] = pa.Column(pa.Float, nullable=df[col].isnull().any())
        elif pd.api.types.is_bool_dtype(dtype):
            col_map[col] = pa.Column(pa.Bool, nullable=df[col].isnull().any())
        else:
            col_map[col] = pa.Column(pa.String, nullable=df[col].isnull().any())

    schema = DataFrameSchema(col_map)
    print('Inferred pandera schema:')
    print(schema)
    # run validation (this will raise if invalid)
    validated_df = schema.validate(df, lazy=True)
    print('Validation passed. Number of rows:', len(validated_df))


# MODELLING

## 16. Load cleaned data

In [ ]:
# 1. Load cleaned data
df = pd.read_csv(CLEAN_PATH)
print('Loaded shape:', df.shape)
display(df.head())

# 17. Prepare features and target

In [ ]:

TARGET = 'target'
if TARGET not in df.columns:
    raise KeyError('Target column not found in cleaned data.')
X = df.drop(columns=[TARGET])
y = df[TARGET]

# Identify numeric and categorical columns simply
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
print('Numeric cols:', num_cols)
print('Categorical cols:', cat_cols)


## 18. Train/test split
We keep a held-out test set for final evaluation.

For professional healthcare machine learning applications, the optimal approach employs an 80-20 train-test split with mandatory stratification and a fixed random state, as this configuration ensures maximal training data utilization for model development while reserving sufficient samples for clinically meaningful validation; crucially, stratification preserves the real-world prevalence of medical conditions across both partitions, preventing skewed performance metrics that could lead to dangerous clinical misinterpretations, and the fixed random state guarantees reproducible research outcomes essential for regulatory compliance and patient safety in healthcare environments where model reliability directly impacts diagnostic accuracy and treatment decisions.



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)

## 19. Preprocessing pipelines
Numeric: median imputation + standard scaling. Categorical: mode imputation + one-hot encoding.

This preprocessing pipeline systematically transforms raw data into machine-learning-ready format by creating separate processing streams for numerical and categorical variables—handling missing values, scaling numerical features, and encoding categories—then seamlessly combines them into a unified feature set that preserves data integrity while making it compatible with ML algorithms.



In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])
print('Preprocessor ready')

# 20. Baseline models
We'll train Logistic Regression (with class weight) and Random Forest. We'll also try a SMOTE pipeline.

We are utilizing the our because:

Logistic Regression → simple, interpretable baseline for binary classification.

Random Forest → strong nonlinear model, robust and balanced.

SMOTE + RF → fixes class imbalance effectively.

XGBoost (Optuna) → powerful tuned model, often top performer for tabular data.

## 20.1. Logistic Regression with class weighting

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

lr_pipe = Pipeline([
    ('pre', preprocessor),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42))
])
lr_pipe.fit(X_train, y_train)
lr_probs = lr_pipe.predict_proba(X_test)[:,1]
print('Logistic AUC:', roc_auc_score(y_test, lr_probs))
# Random Forest without SMOTE
rf_pipe = Pipeline([
    ('pre', preprocessor),
    ('clf', RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced'))
])
rf_pipe.fit(X_train, y_train)
rf_probs = rf_pipe.predict_proba(X_test)[:,1]
print('Random Forest AUC:', roc_auc_score(y_test, rf_probs))

## 21. SMOTE pipeline (oversampling) + Random Forest
SMOTE operates only on numeric arrays; using imbalanced-learn's pipeline simplifies this.


In [ ]:
smote = SMOTE(random_state=42)
smote_pipe = ImbPipeline([
    ('pre', preprocessor),
    ('smote', smote),
    ('clf', RandomForestClassifier(n_estimators=200, random_state=42))
])
smote_pipe.fit(X_train, y_train)
smote_probs = smote_pipe.predict_proba(X_test)[:,1]
print('SMOTE + RF AUC:', roc_auc_score(y_test, smote_probs))


## 21. Evaluate and plot ROC and PR curves for the models


In [ ]:
models = {
    'Logistic': (lr_pipe, lr_probs),
    'RandomForest': (rf_pipe, rf_probs),
    'SMOTE_RandomForest': (smote_pipe, smote_probs)
}
plt.figure(figsize=(8,6))
for name, (m, probs) in models.items():
    fpr = np.nan
    try:
        from sklearn.metrics import roc_curve
        fpr, tpr, _ = roc_curve(y_test, probs)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{name} (AUC={roc_auc:.3f})")
    except Exception as e:
        print('Could not plot ROC for', name, e)
plt.plot([0,1],[0,1],'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend()
fn = os.path.join(IMAGES_DIR, 'roc_curves.png')
plt.savefig(fn, dpi=150, bbox_inches='tight')
plt.show()

# Precision-Recall
plt.figure(figsize=(8,6))
for name, (m, probs) in models.items():
    try:
        precision, recall, _ = precision_recall_curve(y_test, probs)
        pr_auc = auc(recall, precision)
        plt.plot(recall, precision, label=f"{name} (PR-AUC={pr_auc:.3f})")
    except Exception as e:
        print('Could not plot PR for', name, e)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend()
fn = os.path.join(IMAGES_DIR, 'pr_curves.png')
plt.savefig(fn, dpi=150, bbox_inches='tight')
plt.show()


## 22. Confusion matrix & classification report for the best model (by AUC)


In [ ]:
aucs = {name: roc_auc_score(y_test, probs) for name, (m, probs) in models.items()}
best_name = max(aucs, key=aucs.get)
print('AUCs:', aucs)
print('Best model by AUC:', best_name)
best_model = models[best_name][0]
best_probs = models[best_name][1]
best_preds = (best_probs >= 0.5).astype(int)
cm = confusion_matrix(y_test, best_preds)
print('Confusion matrix:\n', cm)
print('\nClassification report:')
print(classification_report(y_test, best_preds))
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title(f'Confusion Matrix - {best_name}')
fn = os.path.join(IMAGES_DIR, f'confusion_matrix_{best_name}.png')
plt.savefig(fn, dpi=150, bbox_inches='tight')
plt.show()


## 23. Hyperparameter tuning with Optuna for XGBoost
We will run a simple Optuna search to maximize cross-validated AUC. This is beginner-friendly but still powerful.


In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 400),
        'max_depth': trial.suggest_int('max_depth', 2, 8),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'use_label_encoder': False,
        'eval_metric': 'logloss',
        'random_state': 42
    }
    model = Pipeline([
        ('pre', preprocessor),
        ('clf', XGBClassifier(**params))
    ])
    # cross-validated AUC (stratified)
    scores = cross_val_score(model, X_train, y_train, cv=StratifiedKFold(n_splits=4, shuffle=True, random_state=42), scoring='roc_auc')
    return float(np.mean(scores))

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)
print('Best trial:')
print(study.best_trial.params)


# 24. Train a final XGBoost model with the best params

In [ ]:
# Train a final XGBoost model with the best params
best_params = study.best_trial.params
best_params.update({'use_label_encoder': False, 'eval_metric': 'logloss', 'random_state': 42})
final_xgb = Pipeline([
    ('pre', preprocessor),
    ('clf', XGBClassifier(**best_params))
])
final_xgb.fit(X_train, y_train)
xgb_probs = final_xgb.predict_proba(X_test)[:,1]
print('Final XGB AUC:', roc_auc_score(y_test, xgb_probs))
# Save model
# Ensure the directory exists before saving
models_dir = os.path.join('/mnt/data/heart-disease-project', 'models')
os.makedirs(models_dir, exist_ok=True)
joblib.dump(final_xgb, os.path.join(models_dir, 'final_xgb.pkl'))
print('Saved final model to:', os.path.join(models_dir, 'final_xgb.pkl'))

## 10. Compare final XGBoost to previous best and save results


In [ ]:
finals = {
    'XGBoost_Optuna': (final_xgb, xgb_probs)
}
all_models = {**models, **finals}
for name, (m, probs) in all_models.items():


    try:
        precision, recall, _ = precision_recall_curve(y_test, probs)
        print(f"{name}: AUC={roc_auc_score(y_test, probs):.3f}, PR-AUC={auc(recall, precision):.3f}")
    except Exception as e:
        print('Skipped metrics for', name, e)

In [ ]:
# Save a simple CSV of model scores
scores = []
for name, (m, probs) in all_models.items():
    try:
        scores.append({'model': name, 'auc': roc_auc_score(y_test, probs)})
    except:
        pass
scores_df = pd.DataFrame(scores).sort_values('auc', ascending=False)
# Ensure the reports directory exists before saving
reports_dir = os.path.join('/mnt/data/heart-disease-project', 'reports')
os.makedirs(reports_dir, exist_ok=True)
scores_df.to_csv(os.path.join(reports_dir, 'model_scores.csv'), index=False)
display(scores_df)

## 25. Save ROC curve for final XGBoost


In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test, xgb_probs)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f'XGBoost (AUC={roc_auc_score(y_test, xgb_probs):.3f})')
plt.plot([0,1],[0,1],'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Final XGBoost ROC')
plt.legend()
fn = os.path.join(IMAGES_DIR, 'final_xgb_roc.png')
plt.savefig(fn, dpi=150, bbox_inches='tight')
plt.show()


## 26. Save Precision-Recall for final XGBoost


In [ ]:
precision, recall, _ = precision_recall_curve(y_test, xgb_probs)
plt.figure(figsize=(6,5))
plt.plot(recall, precision, label=f'XGBoost (PR-AUC={auc(recall, precision):.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Final XGBoost Precision-Recall')
plt.legend()
fn = os.path.join(IMAGES_DIR, 'final_xgb_pr.png')
plt.savefig(fn, dpi=150, bbox_inches='tight')
plt.show()


## 27. Feature importance (from XGBoost) — simple approach
We will extract feature names after preprocessing by applying the preprocessor to a sample and then retrieving feature names.


In [ ]:
from sklearn.exceptions import NotFittedError
# from sklearn.preprocessing import OneHotEncoder, StandardScaler # Already imported earlier
# from sklearn.impute import SimpleImputer # Already imported earlier


def get_feature_names(column_transformer):
    """Get feature names from a ColumnTransformer after fitting."""
    output_features = []
    for name, transformer, original_features in column_transformer.transformers_:
        # Skip if the list of original features for this transformer is empty
        if not original_features:
            print(f"Info: Skipping transformer '{name}' as it is applied to an empty list of columns.")
            continue

        if transformer == 'passthrough':
            output_features.extend(original_features)
        elif hasattr(transformer, 'get_feature_names_out'):
            # For transformers with get_feature_names_out (like OneHotEncoder, StandardScaler, etc.)
            try:
                output_features.extend(transformer.get_feature_names_out(original_features))
            except NotFittedError:
                 # If the transformer isn't fitted yet, fallback to original names as a warning
                 print(f"Warning: Transformer {name} is not fitted. Using original column names: {original_features}")
                 output_features.extend(original_features)
            except Exception as e:
                 print(f"Warning: Could not get feature names from transformer {name} using get_feature_names_out: {e}. Using original column names.")
                 output_features.extend(original_features)
        elif hasattr(transformer, 'get_feature_names'):
             # For older transformers with get_feature_names
            try:
                output_features.extend(transformer.get_feature_names(original_features))
            except NotFittedError:
                 print(f"Warning: Transformer {name} is not fitted. Using original column names: {original_features}")
                 output_features.extend(original_features)
            except Exception as e:
                 print(f"Warning: Could not get feature names from transformer {name} using get_feature_names: {e}. Using original column names.")
                 output_features.extend(original_features)

        elif hasattr(transformer, 'named_steps'):
            # For pipelines, try to get names from the last step if it has get_feature_names_out
            last_step = list(transformer.named_steps.values())[-1]
            if hasattr(last_step, 'get_feature_names_out'):
                try:
                    output_features.extend(last_step.get_feature_names_out(original_features))
                except NotFittedError:
                    print(f"Warning: Last step of pipeline {name} is not fitted. Using original column names: {original_features}")
                    output_features.extend(original_features)
                except Exception as e:
                    print(f"Warning: Could not get feature names from last step of pipeline {name} using get_feature_names_out: {e}. Using original column names.")
                    output_features.extend(original_features)
            elif hasattr(last_step, 'get_feature_names'):
                try:
                    output_features.extend(last_step.get_feature_names(original_features))
                except NotFittedError:
                    print(f"Warning: Last step of pipeline {name} is not fitted. Using original column names: {original_features}")
                    output_features.extend(original_features)
                except Exception as e:
                    print(f"Warning: Could not get feature names from last step of pipeline {name} using get_feature_names: {e}. Using original column names.")
                    output_features.extend(original_features)
            else:
                # Fallback for pipelines where the last step doesn't have feature names method
                # If the last step is a transformer that doesn't change the number of features (like imputer or scaler),
                # we can often assume the original features are the output features for this step of the CT.
                if isinstance(last_step, (SimpleImputer, StandardScaler)):
                     print(f"Info: Using original features for pipeline step {name} (likely Imputer/Scaler).")
                     output_features.extend(original_features)
                else:
                     print(f"Warning: Could not get detailed feature names from pipeline step {name}. Using original column names.")
                     output_features.extend(original_features)
        else:
            # Fallback for simple transformers without specific methods
            print(f"Warning: Could not get detailed feature names from transformer {name}. Using original column names.")
            output_features.extend(original_features)

    return output_features

# Fit the preprocessor before getting feature names
# If preprocessor is already part of a fitted pipeline, this fit might not be strictly necessary
# for accessing _fitted_ transformers, but it ensures the preprocessor itself is fitted.
try:
    preprocessor.fit(X_train)
except Exception as e:
    # This might happen if the preprocessor is already fitted within a pipeline
    print("Preprocessor might be already fitted within the model pipeline:", e)
    pass # Assume preprocessor is fitted within the final_xgb pipeline


# Get feature names from the fitted preprocessor within the final model pipeline
# Access the fitted preprocessor from the final model pipeline
fitted_preprocessor = final_xgb.named_steps['pre']
feat_names = get_feature_names(fitted_preprocessor)


try:
    booster = final_xgb.named_steps['clf']
    # xgboost feature importance uses original feature indices, but with pipeline we have transformed features
    importances = booster.feature_importances_

    # Ensure the number of feature importances matches the number of feature names
    if len(importances) != len(feat_names):
        print(f"Error: Mismatch between number of feature importances ({len(importances)}) and feature names ({len(feat_names)}).")
        # Fallback to simpler naming or raise an error
        fi_df = pd.DataFrame({'feature': [f'feature_{i}' for i in range(len(importances))], 'importance': importances})
    else:
        fi_df = pd.DataFrame({'feature': feat_names, 'importance': importances})

    fi_df = fi_df.sort_values('importance', ascending=False).head(20)
    plt.figure(figsize=(8,6))
    sns.barplot(x='importance', y='feature', data=fi_df)
    plt.title('Top 20 feature importances (XGBoost)')
    fn = os.path.join(IMAGES_DIR, 'xgb_feature_importance.png')
    plt.savefig(fn, dpi=150, bbox_inches='tight')
    plt.show()
except Exception as e:
    print('Could not compute feature importances or plot:', e)

## 28. Save final model and report


In [ ]:
# Define the base directory for saving models and reports
base = '/content/gdrive/MyDrive/heart-disease-project'

os.makedirs(os.path.join(base, 'models'), exist_ok=True)
joblib.dump(final_xgb, os.path.join(base, 'models', 'final_xgb_optuna.pkl'))
print('Saved final model to models/final_xgb_optuna.pkl')
print('Modeling notebook complete. Review images in images/ and model scores in reports/model_scores.csv')

# 29 - Interpretation

This notebook explains how the best model makes its predictions.
- Uses **SHAP** for global + local feature importance.
- Optionally uses **LIME** for local instance interpretation.
- Saves interpretability plots to `/images/`.


In [ ]:
# --- Imports ---
import pandas as pd
import shap
import joblib
import matplotlib.pyplot as plt
from lime.lime_tabular import LimeTabularExplainer
from sklearn.metrics import roc_auc_score

# Load processed data and model
df = pd.read_csv('../data/processed/heart_clean.csv')
model = joblib.load('../models/final_xgb_optuna.pkl')

X = df.drop(columns=['target'])
y = df['target']
print('Data shape:', X.shape)


In [ ]:
!pip install lime

In [ ]:
# --- SHAP Interpretation ---
# SHAP helps explain global feature importance and per-patient explanations.
explainer = shap.Explainer(model, X)
shap_values = explainer(X)

# Global summary plot
plt.title('SHAP Summary Plot')
shap.summary_plot(shap_values, X, show=False)
plt.savefig('../images/shap_summary.png', bbox_inches='tight')
plt.show()

# Bar plot (mean absolute value)
shap.summary_plot(shap_values, X, plot_type='bar', show=False)
plt.title('Mean Absolute SHAP Values')
plt.savefig('../images/shap_importance_bar.png', bbox_inches='tight')
plt.show()


In [ ]:
# --- Dependence Plot for Top Features ---
top_feature = X.columns[0]
shap.dependence_plot(top_feature, shap_values.values, X, show=False)
plt.title(f'SHAP Dependence for {top_feature}')
plt.savefig(f'../images/shap_dependence_{top_feature}.png', bbox_inches='tight')
plt.show()


In [ ]:
# --- LIME Example (optional, for one patient) ---
import numpy as np
explainer_lime = LimeTabularExplainer(
    training_data=np.array(X),
    feature_names=X.columns,
    class_names=['No Disease', 'Disease'],
    mode='classification'
)

# Pick one random patient
i = np.random.randint(0, len(X))
exp = explainer_lime.explain_instance(X.iloc[i].values, model.predict_proba)
exp.save_to_file('../images/lime_patient_example.html')
print(f'LIME explanation saved for patient #{i}')


In [ ]:
# --- Fairness / Subgroup Comparison (if demographic columns exist) ---
if any(col in X.columns for col in ['sex', 'gender', 'age']):
    print('Running subgroup comparison...')
    if 'sex' in X.columns:
        male_auc = roc_auc_score(y[X['sex']==1], model.predict_proba(X[X['sex']==1])[:,1])
        female_auc = roc_auc_score(y[X['sex']==0], model.predict_proba(X[X['sex']==0])[:,1])
        print(f'AUC Male: {male_auc:.3f}, Female: {female_auc:.3f}')
    if 'age' in X.columns:
        median_age = X['age'].median()
        young_auc = roc_auc_score(y[X['age']<=median_age], model.predict_proba(X[X['age']<=median_age])[:,1])
        old_auc = roc_auc_score(y[X['age']>median_age], model.predict_proba(X[X['age']>median_age])[:,1])
        print(f'AUC Younger vs Older: {young_auc:.3f} / {old_auc:.3f}')


In [ ]:
# --- Summary of Insights ---
print('\nKey Insights:')
print('- Features like age, cholesterol, and chest pain type typically show strong SHAP values.')
print('- Positive SHAP value → higher heart disease risk contribution.')
print('- LIME helps explain individual predictions in an easy-to-read format.')
print('- Fairness metrics suggest whether the model behaves similarly across groups.')


# 30. SHAP explainability (global + local)
This cell assumes you have a fitted pipeline named `clf` and `X_train`/`X_test`.

In [ ]:

# SHAP analysis (global + local)
import shap
import numpy as np

# Ensure model exists
if 'clf' not in globals():
    print('Warning: clf not found. Please load or train your pipeline and ensure it is named `clf`.')
else:
    model = clf.named_steps.get('model', None)
    preprocessor = clf.named_steps.get('preprocessor', None)
    if model is None or preprocessor is None:
        print('clf does not have expected steps preprocessor/model.')
    else:
        # Use a small sample for SHAP to keep runtime low
        X_sample = X_train.sample(n=min(200, len(X_train)), random_state=RANDOM_STATE)
        X_pre = preprocessor.transform(X_sample)
        try:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_pre)
            try:
                shap.summary_plot(shap_values, X_pre)
            except Exception:
                shap.summary_plot(shap_values, X_pre, feature_names=getattr(preprocessor, 'get_feature_names_out', None) and preprocessor.get_feature_names_out())
        except Exception as e:
            print('TreeExplainer failed, falling back to KernelExplainer:', e)
            try:
                background = X_pre[:50]
                explainer = shap.KernelExplainer(lambda x: model.predict_proba(x)[:,1] if hasattr(model, 'predict_proba') else model.predict(x), background)
                shap_values = explainer.shap_values(X_pre[:50])
                shap.summary_plot(shap_values, X_pre[:50])
            except Exception as e2:
                print('KernelExplainer also failed:', e2)
